In [2]:
import os
import time
import re
import requests
from bs4 import BeautifulSoup

import numpy as np
import matplotlib.pyplot as plt

import japanize_matplotlib
from astropy.io import fits
from astropy.utils.data import download_file

d_year = '2022'
d_date = '20221107'

url_base = 'https://solarwww.mtk.nao.ac.jp/mitaka_solar/ha-mtkft1/fts/'

url = url_base + d_year + '/' + d_date + '/'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
get_list = soup.find_all('a')

pattern = 'mtk_ft_ha_fd_\d{8}T\d{6}_i_000_m.fts.gz'

prog = re.compile(pattern)

figsize_px = np.array([2048, 2048])
dpi = 300
figsize_inch = figsize_px / dpi

fig = plt.figure(dpi=dpi, figsize=(figsize_inch), facecolor='lightslategray')

png_no = 0

filepath = './image/' + d_date + '/'
os.makedirs(filepath, exist_ok=True)

for i, fits_list in enumerate(get_list):
    
    fits_href = fits_list.attrs['href']

    if prog.match(fits_href):

        png_no += 1

        # sleep
        time.sleep(1)
        print(png_no, fits_list)

        image_file = download_file(url + fits_href, cache=True )
        hdu_list = fits.open(image_file)
        image_header = hdu_list[0].header
        image_data = hdu_list[0].data
        hdu_list.close()
        #image_data = fits.getdata(image_file)

        image_dt_target = 'T'
        image_date_time = image_header['DATE_OBS']
        dtidx = image_date_time.find(image_dt_target)
        image_date = image_date_time[:dtidx]
        image_time = image_date_time[dtidx+len(image_dt_target):]

        image_UT_target = '.'
        UTidx = image_time.find(image_UT_target)
        image_time_UT = image_time[:UTidx] + ' UT'

        save_date = image_date[:4] + image_date[5:7] + image_date[8:]
        save_time = image_time[:2] + image_time[3:5] + image_time[6:8]

        png_no_str = str(png_no)

        plt.axis("off")

        plt.text(15, 1980, image_header['OBJECT'] + ' / ' + image_header['WAVELNTH']+ ' / ' + image_header['TYPE_OBS'], color='whitesmoke')
        plt.text(15, 15, image_date + ' ' + image_time_UT, color='whitesmoke')
        plt.text(1620, 15, 'Credit: NAOJ', color='whitesmoke', size='small')

        #plt.imshow(image_data, cmap='hot', origin='lower')
        plt.imshow(image_data, origin='lower')
        #plt.imshow(image_data, cmap='gray', origin='lower')

        plt.savefig(filepath + 'im_' + png_no_str + '_' + save_date + '_' + save_time + '.png', format='png', dpi=dpi)

        plt.clf()

    if i == 20:
        break

plt.close()


1 <a href="mtk_ft_ha_fd_20221106T232508_i_000_m.fts.gz"> mtk_ft_ha_fd_20221106T232508_i_000_m.fts.gz</a>
2 <a href="mtk_ft_ha_fd_20221106T232538_i_000_m.fts.gz"> mtk_ft_ha_fd_20221106T232538_i_000_m.fts.gz</a>
3 <a href="mtk_ft_ha_fd_20221106T232608_i_000_m.fts.gz"> mtk_ft_ha_fd_20221106T232608_i_000_m.fts.gz</a>
4 <a href="mtk_ft_ha_fd_20221106T232638_i_000_m.fts.gz"> mtk_ft_ha_fd_20221106T232638_i_000_m.fts.gz</a>
5 <a href="mtk_ft_ha_fd_20221106T232708_i_000_m.fts.gz"> mtk_ft_ha_fd_20221106T232708_i_000_m.fts.gz</a>
